**Objective**: Down-month reduction and return maximization under discrete liquidity and inertia constraints.

# 1. Mathematical Objective Function
To achieve the dual goal of maximizing returns while specifically penalizing "down months," we utilize a Lower Partial Moment (LPM) framework.

Target Function:

$\(\max _{w}\quad \Phi (w)=\underbrace{E[R_{p}]}_{\text{Projected\ Return}}-\lambda \cdot \underbrace{\frac{1}{T}\sum _{t=1}^{T}\min (0,R_{p,t})^{2}}_{\text{Semi-Variance\ (Downside\ Risk)}}\)$

$\(R_{p,t}\)$: Portfolio return at time $\(t\)$, calculated as $\(\sum w_{i}R_{i,t}\)$.

$\(\lambda \)$: Risk-aversion coefficient (higher values prioritize avoiding down months).

# 2. Constraints Specification

Allocation Floor: $\(w_{i,t}\in \{0\}\cup [0.02,1.0]\)$. Assets must have at least 2% weight or be excluded entirely.

Minimum Trade Step: $\(|w_{i,t}-w_{i,t-1}|\in \{0\}\cup [0.01,1.0]\)$. Changes to any weight must be at least 1%.

Inertia (Lock-up): If $\(w_{i,t}\ne w_{i,t-1}\), then \(w_{i,t}=w_{i,t+1}=w_{i,t+2}\)$. This enforces a 3-month holding period for any modified weight.

# 3. Suggested Numerical Optimizers

1. Genetic Algorithms (PyGAD / DEAP): Excellent for handling "if-else" constraints and non-continuous search spaces.

2. Particle Swarm Optimization (PySwarms): Good for balancing the exploration of weight combinations under the 3-month inertia constraint.

3. Differential Evolution (Used below): Best for non-convex spaces with discrete "jumps".

# 4. Python Implementation

To solve this, I will use a Genetic Algorithm approach. Standard solvers struggle with your specific constraints—particularly the "all-or-nothing" 1% minimum change and the 3-month inertia rule, which create a non-convex, discontinuous search space.
I have structured the code into two main functions as requested. We will use a "sliding window" logic to simulate the walk-forward optimization.


In [5]:
import pandas as pd
import numpy as np
from scipy.optimize import differential_evolution

In [ ]:
def optimize_portfolio_history(returns_df, initial_weights_df, lambda_risk=0.5):
    """
    Outputs: Dataframe with optimized weights for each month.
    Handles: 2% min weight, 1% min change, and 3-month inertia.
    """
    managers = returns_df.columns[1:]
    months = returns_df['month'].iloc[::-1].reset_index(drop=True) # Work chronological
    returns_data = returns_df.set_index('month').iloc[::-1]
    
    # Initialize tracking
    current_weights = initial_weights_df.set_index('fund manager')['current weight'].values
    history_weights = []
    months_since_change = np.array([3] * len(managers)) # Start ready to trade

    for i in range(len(months)):
        # Objective: Maximize return - Penalty for down months (Semi-variance)
        def objective(w):
            # Normalize to sum to 1
            w = w / np.sum(w)
            # Use data up to current month for expectation
            hist = returns_data.iloc[:i+1].fillna(0)
            port_return = np.dot(hist, w)
            proj_return = np.mean(port_return)
            downside = np.mean(np.square(np.minimum(port_return, 0)))
            return -(proj_return - lambda_risk * downside)

        # Constraints defined as bounds and penalty logic
        bounds = [(0, 1.0) for _ in range(len(managers))]
        
        # Optimize using Differential Evolution (robust for discrete-like constraints)
        result = differential_evolution(objective, bounds, tol=0.01)
        raw_weights = result.x / np.sum(result.x)
        
        # Apply Logic Constraints: 1% min change & 2% min weight & 3-month rule
        new_weights = np.copy(current_weights)
        for j in range(len(managers)):
            proposed_change = abs(raw_weights[j] - current_weights[j])
            
            # Condition: If we are allowed to change (3 months passed) 
            # AND the change is significant (>1%)
            if months_since_change[j] >= 3 and proposed_change >= 0.01:
                # Apply 2% floor or 0
                final_w = raw_weights[j] if raw_weights[j] >= 0.02 else 0.0
                if abs(final_w - current_weights[j]) >= 0.01:
                    new_weights[j] = final_w
                    months_since_change[j] = 0
            
            months_since_change[j] += 1
            
        # Re-normalize to ensure sum is 1.0 after logic application
        new_weights = new_weights / np.sum(new_weights)
        current_weights = new_weights
        history_weights.append(list(current_weights))

    weight_df = pd.DataFrame(history_weights, columns=managers)
    weight_df.insert(0, 'month', months)
    return weight_df

In [ ]:
def calculate_performance_comparison(returns_df, optimized_weights_df, initial_weights_df):
    """
    Outputs: Comparison dataframe with returns and down-month indicators.
    """
    # Align data
    returns_data = returns_df.set_index('month').iloc[::-1].fillna(0)
    opt_weights = optimized_weights_df.set_index('month')
    
    # Original Return (using the static initial weights provided)
    orig_w = initial_weights_df.set_index('fund manager')['current weight'].values
    orig_returns = returns_data.dot(orig_w)
    
    # Optimized Return
    opt_returns = (returns_data * opt_weights).sum(axis=1)
    
    comparison = pd.DataFrame({
        'original_return': orig_returns,
        'optimized_return': opt_returns
    })
    
    comparison['original_down'] = (comparison['original_return'] < 0).astype(int)
    comparison['optimized_down'] = (comparison['optimized_return'] < 0).astype(int)
    
    return comparison.reset_index()

In [ ]:
# 1. Load Data
df_returns = pd.read_excel('base.xlsx', sheet_name='ORG DATA')
df_weights = pd.read_excel('base.xlsx', sheet_name='org weight')

# 2. Run Optimization
# Note: lambda_risk balances return vs. down-month protection
df_opt_weights = optimize_portfolio_history(df_returns, df_weights, lambda_risk=0.7)

# 3. Generate Performance Table
df_performance = calculate_performance_comparison(df_returns, df_opt_weights, df_weights)

# Display Results
print(df_performance.head())